Problem description: The goal is to design an optimized weekly meal plan that balances cooking time, ingredient availability, and variety. The meal plan should: minimize the total time spent cooking each day, adhering to daily time constraints, prioritize recipes that use ingredients already available in the inventory to reduce waste, ensure a diverse menu and favor recipes that utilize a specific prioritized ingredient to accomodate for preferences.

In [ ]:
!pip install pulp
from pulp import *

In [ ]:
#days and recipes

days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
recipes = ["Grilled Cheese", "Frozen Pizza", "Quesadilla", "Soba Noodles", "Shrimp Alfredo", "Chicken Stir Fry", "Salmon and Orzo", "Marry Me Chicken", "Baked Feta Pasta", "Mashed Potatoes", "French Onion Pasta", "Burritos"]

#cooking times in minutes

cooking_times = {
    "Grilled Cheese": 10,
    "Frozen Pizza": 10,
    "Quesadilla": 10,
    "Soba Noodles": 20,
    "Shrimp Alfredo": 25,
    "Chicken Stir Fry": 30,
    "Salmon and Orzo": 40,
    "Marry Me Chicken": 40,
    "Baked Feta Pasta": 45,
    "Mashed Potatoes": 60,
    "French Onion Pasta": 50,
    "Burritos": 35
}

#ingredient usage per recipe

ingredients = {
    "Bread": {"Grilled Cheese": 2}, #units
    "Deli Meats": {"Grilled Cheese": 2},
    "Toast Cheese": {"Grilled Cheese": 1, "Quesadillas": 2},
    "Tortillas": {"Qesadilla": 1, "Burritos": 8},
    "Frozen Pizza Box": {"Frozen Pizza": 1},
    "Soba Noodle Pack": {"Soba Noodles": 2},
    "Soy Sauce": {"Soba Noodles": 80 ,"Chicken Stir Fry": 80}, #ml
    "Sesame Oil": {"Soba Noodles": 45,"Chicken Stir Fry": 10}, #ml
    "Sesame Seeds": {"Soba Noodles": 30, "Chicken Stir Fry": 15}, #g
    "Green Onions": {"Soba Noodles": 250}, #g
    "Pasta": {"Shrimp Alfredo": 450, "Baked Feta Pasta": 280, "French Onion Pasta": 340}, #g
    "Butter": {"Shrimp Alfredo": 45, "Mashed Potatoes": 40, "French Onion Pasta": 45}, #g
    "Shrimp": {"Shrimp Alfredo": 450}, #g
    "Garlic": {"Shrimp Alfredo": 2,"Chicken Stir Fry": 3, "Salmon annd Orzo": 2, "Marry Me chicken": 1, "Baked Feta Pasta": 3, "French Onion Pasta": 4, "Burritos": 2}, #cloves
    "Flour": {"Shrimp Alfredo": 30}, #g
    "Heavy Cream": {"Shrimp Alfredo": 240, "Salmon annd Orzo": 40, "Marry Me chicken": 120}, #ml
    "Milk": {"Shrimp Alfredo": 120, "Mashed Potatoes": 100}, #ml
    "Egg Yolk": {"Shrimp Alfredo": 1},
    "Parmesan": {"Shrimp Alfredo": 100, "Marry Me chicken": 15}, #g
    "Boullion Cube": {"Chicken Stir Fry": 0.5, "Salmon annd Orzo": 1, "Marry Me chicken": 0.5, "French Onion Pasta": 4},
    "Honey": {"Chicken Stir Fry": 45}, #ml
    "Chicken Breast": {"Chicken Stir Fry": 450, "Marry Me chicken": 225, "Mashed Potatoes": 300}, #g
    "Broccoli": {"Chicken Stir Fry": 2}, #heads of broccoli
    "Carrots": {"Chicken Stir Fry": 200, "Mashed Potatoes": 200}, #g
    "Salmon Fillets": {"Salmon annd Orzo": 170}, #g
    "Olive Oil": {"Salmon annd Orzo": 22, "Marry Me chicken": 23, "Baked Feta Pasta": 120}, #ml
    "Orzo": {"Salmon annd Orzo": 115}, #g
    "Sun Dried Tomatoes": {"Salmon annd Orzo": 40, "Marry Me chicken": 60}, #g
    "Onion": {"Salmon annd Orzo": 0.5, "Baked Feta Pasta": 0.5, "French Onion Pasta": 3, "Burritos": 120}, #unit
    "Feta Cheese": {"Baked Feta Pasta": 1},
    "Cherry Tomatoes": {"Baked Feta Pasta": 900}, #g
    "Potatoes": {"Mashed Potatoes": 400}, #g
    "Gruyere Cheese": {"French Onion Pasta": 100}, #g
    "Jalapenos": {"Burritos": 1}, #unit
    "Beef": {"Burritos": 450}, #g
    "Canned Beans": {"Burritos": 2},
    "Cheddar Cheese": {"Burritos": 115}, #g
    "Rice": {"Burritos": 480}, #g

}

#inventory

inventory = {
    "Bread": 8, #slices
    "Deli Meats": 6,
    "Toast Cheese": 5,
    "Tortillas": 0,
    "Frozen Pizza Box": 1,
    "Soba Noodle Pack": 4,
    "Soy Sauce": 150, #ml
    "Sesame Oil": 100, #ml
    "Sesame Seeds": 100, #g
    "Green Onions": 0, #g
    "Pasta": 3000, #g
    "Butter": 250, #g
    "Shrimp": 200, #g
    "Garlic": 2, #cloves
    "Flour": 1000,
    "Heavy Cream": 480, #ml
    "Milk": 1000, #ml
    "Egg Yolk": 4,
    "Parmesan": 0, #g
    "Boullion Cube": 5, #unit
    "Honey": 200, #ml
    "Chicken Breast": 0, #g
    "Broccoli": 0, #heads of broccoli
    "Carrots": 6, #g
    "Salmon Fillets": 0, #g
    "Olive Oil": 40, #ml
    "Orzo": 0,
    "Sun Dried Tomatoes": 0, #g
    "Onion": 2, #unit
    "Feta Cheese": 0,
    "Cherry Tomatoes": 200, #g
    "Potatoes": 100, #g
    "Gruyere Cheese": 500, #g
    "Jalapenos": 0, #unit
    "Beef": 300, #g
    "Canned Beans": 1,
    "Cheddar Cheese": 0, #g
    "Rice": 1000, #g
}

waste_penalty = {
    "Bread": 1.0,
    "Deli Meats": 1.5,
    "Toast Cheese": 1.0,
    "Tortillas": 0.5,
    "Frozen Pizza Box": 0.1,
    "Soba Noodle Pack": 0.1,
    "Soy Sauce": 0.1,
    "Sesame Oil": 0.1,
    "Sesame Seeds": 0.1,
    "Green Onions": 1.5,
    "Pasta": 0.1,
    "Butter": 0.3,
    "Shrimp": 1.5,
    "Garlic": 0.5,
    "Flour": 0.1,
    "Heavy Cream": 2.0,
    "Milk": 1.5,
    "Egg Yolk": 2.0,
    "Parmesan": 1.5,
    "Boullion Cube": 0.1,
    "Honey": 0.1,
    "Chicken Breast": 3.0,
    "Broccoli": 1.5,
    "Carrots": 0.8,
    "Salmon Fillets": 3.5,
    "Olive Oil": 0.1,
    "Orzo": 0.1,
    "Sun Dried Tomatoes": 1.0,
    "Onion": 0.5,
    "Feta Cheese": 2.0,
    "Cherry Tomatoes": 1.5,
    "Potatoes": 0.5,
    "Gruyere Cheese": 2.0,
    "Jalapenos": 1.0,
    "Beef": 2.5,
    "Canned Beans": 0.1,
    "Cheddar Cheese": 1.5, #g
    "Rice": 0.1, #g
}




In [ ]:
prob = LpProblem("Meal_Plan_Optimization", LpMinimize)

#decision variables

x = LpVariable.dicts("Cook", [(day, recipe) for day in days for recipe in recipes], cat="Binary")

#maximum cooking time per day
max_time_per_day = {
    "Monday": 60,
    "Tuesday": 50,
    "Wednesday": 20,
    "Thursday": 50,
    "Friday": 20
}

#prioritized ingredient (can change)
priority_ingredient = "Gruyere Cheese"
priority_weight = 100  # High weight to prioritize recipes using this ingredient


ingredient_contribution = {
    recipe: ingredients.get(priority_ingredient, {}).get(recipe, 0) for recipe in recipes
}

overlap_score = {}
for recipe in recipes:
    overlap_score[recipe] = sum(
        1 for ing, usage in ingredients.items() if recipe in usage and inventory.get(ing, 0) >= usage[recipe]
    )


#objective function: minimize cooking time while prioritizing the specific ingredient and ingredient overlap

overlap_weight = 10
prob += (
    lpSum(cooking_times[recipe] * x[(day, recipe)] for day in days for recipe in recipes)  # Minimize time
    - overlap_weight * lpSum(overlap_score[recipe] * x[(day, recipe)] for day in days for recipe in recipes)  # Prioritize overlap
    - priority_weight * lpSum(ingredient_contribution[recipe] * x[(day, recipe)] for day in days for recipe in recipes)  # Prioritize ingredient
)

#constraints

#one recipe per day
for day in days:
  prob += lpSum(x[(day, recipe)] for recipe in recipes) == 1

#cooking time limit
for day in days:
    prob += lpSum(
        cooking_times[recipe] * x[(day, recipe)] for recipe in recipes
    ) <= max_time_per_day[day]

for recipe in recipes:
    prob += lpSum(x[(day, recipe)] for day in days) <= 1


prob.solve()

#results
print("Optimized Meal Plan:")
for var in prob.variables():
  if var.varValue > 0 and "Cook" in var.name:
    print(f"{var.name} = {var.varValue}")


Optimized Meal Plan:
Cook_('Friday',_'Soba_Noodles') = 1.0
Cook_('Monday',_'Chicken_Stir_Fry') = 1.0
Cook_('Thursday',_'Shrimp_Alfredo') = 1.0
Cook_('Tuesday',_'French_Onion_Pasta') = 1.0
Cook_('Wednesday',_'Grilled_Cheese') = 1.0
